In [1]:
#coding=utf-8
from sklearn.metrics import roc_auc_score, recall_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OneHotEncoder
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OneHotEncoder

def load_data():
    df = pd.read_csv("Data/train.csv.gz")
    return df

def eval_matric(y_true, y_prob):
    print("true p ratio", sum(y_true)/ len(y_true))
    y_pred = [1 if i> 0.5 else 0 for i in y_prob]
    print("recall ratio", sum(y_pred)/float(sum(y_true)))

    y_true = np.asarray(y_true)
    y_true = y_true[np.argsort(y_prob)]
    ntrue = 0
    gini = 0
    delta = 0
    n = len(y_true)
    for i in range(n-1, -1, -1):
        y_i = y_true[i]
        ntrue += y_i
        gini += y_i * delta
        delta += 1 - y_i
        gini = 1 - 2 * gini / (ntrue * (n - ntrue))
    print("gini:", gini)
    return gini

In [3]:
print("---loading and preprocessing the data---")
data = load_data()
data = data.set_index("id")
target = data['target']
data.drop(['target'], axis=1, inplace=True)
#data = preprocess(data)

---loading and preprocessing the data---


In [4]:
from util import *

data, sparse_features, dense_features = recognize_feature(data)
print(len(dense_features),len(sparse_features))

sparse_label_dict = dict()
for f in sparse_features:
    sparse_label_dict[f] = data[f].max()

print(data.shape)
#data = one_hot_for_sparse(data, sparse_features)
#data = scalar_for_dense(data, dense_features)
print(data.shape)

sparse : unique sum  376
27 30
(595212, 57)
(595212, 57)


In [5]:
print(sparse_label_dict)

{'ps_ind_01': 7, 'ps_ind_02_cat': 4, 'ps_ind_03': 11, 'ps_ind_04_cat': 2, 'ps_ind_05_cat': 7, 'ps_ind_14': 4, 'ps_ind_15': 13, 'ps_car_01_cat': 12, 'ps_car_02_cat': 2, 'ps_car_03_cat': 2, 'ps_car_04_cat': 9, 'ps_car_05_cat': 2, 'ps_car_06_cat': 17, 'ps_car_07_cat': 2, 'ps_car_08_cat': 1, 'ps_car_09_cat': 5, 'ps_car_10_cat': 2, 'ps_car_11_cat': 103, 'ps_car_11': 4, 'ps_calc_04': 5, 'ps_calc_05': 6, 'ps_calc_06': 10, 'ps_calc_07': 9, 'ps_calc_08': 10, 'ps_calc_09': 7, 'ps_calc_10': 25, 'ps_calc_11': 19, 'ps_calc_12': 10, 'ps_calc_13': 13, 'ps_calc_14': 23}


In [6]:
train, test, y_train, y_test = train_test_split(data, target, test_size=0.33, random_state=42)

In [7]:
train = train.reset_index().drop("id", axis =1)
test = test.reset_index().drop("id", axis =1)
y_train = y_train.reset_index().drop("id", axis =1)
y_test = y_test.reset_index().drop("id", axis =1)

In [13]:
%%time
from Tensorflow.DeepCN import DeepCN
model = DeepCN(sparse_features, dense_features, sparse_label_dict, [2048, 100, 20] , 3, 4)
model.fit(train, y_train, 50, 1000)
metric = model.evaluate(test, y_test, roc_auc_score)
print(metric)

epoch 0:
the times of training is 0, and the loss is 0.60866004
the times of training is 100, and the loss is 0.15554808
the times of training is 200, and the loss is 0.15426256
the times of training is 300, and the loss is 0.1513024
epoch 1:
the times of training is 0, and the loss is 0.1590231
the times of training is 100, and the loss is 0.15548883
the times of training is 200, and the loss is 0.1531513
the times of training is 300, and the loss is 0.1506297
epoch 2:
the times of training is 0, and the loss is 0.15905963
the times of training is 100, and the loss is 0.1554596
the times of training is 200, and the loss is 0.15289032
the times of training is 300, and the loss is 0.15034539
epoch 3:
the times of training is 0, and the loss is 0.15907162
the times of training is 100, and the loss is 0.1552739
the times of training is 200, and the loss is 0.1528381
the times of training is 300, and the loss is 0.15023157
epoch 4:
the times of training is 0, and the loss is 0.15899254
the

%%time
model = XDeepFM(sparse_features, dense_features, sparse_label_dict, [2048, 100, 50] , [25, 20, 20], 3)
model.fit(train, y_train, 100, 1000)
model.evaluate(test, y_test, roc_auc_score)

%%time
from Keras.XDeepFM import XDeepFM
model = XDeepFM(sparse_features, dense_features, sparse_label_dict, [2048, 100, 50] , [25, 20, 20], 3)
model.fit(train, y_train, 50, 1000)
model.evaluate(test, y_test, roc_auc_score)

%%time
from Tensorflow.TFModel import *
model =TFModel(sparse_features, dense_features, sparse_label_dict, [2048, 100, 50] , 3)
model.fit(train, y_train, 100, 1000)
model.evaluate(test, y_test, roc_auc_score)

%%time
from Keras.DeepFM import *
model = DeepFM(sparse_features, dense_features, sparse_label_dict, [2048, 100, 50] , 3)
model.fit(train, y_train, 100, 1000)
model.evaluate(test, y_test, roc_auc_score)

In [9]:
%%time
from Tensorflow.XDeepFM import XDeepFM
model = XDeepFM(sparse_features, dense_features, sparse_label_dict, [2048, 100, 50] , 3)
model.fit(train, y_train, 50, 1000)
metric = model.evaluate(test, y_test, roc_auc_score)
print(metric)

<class 'tensorflow.python.framework.ops.Tensor'>
(?, 3, 30, 30)
(?, 1)
(?, 1)
(?, 1)
(?, 1)
(?, 1)
(?, 1)
(?, 1)
(?, 1)
(?, 1)
(?, 1)
(?, 1)
(?, 1)
(?, 1)
(?, 1)
(?, 1)
(?, 1)
(?, 1)
(?, 1)
(?, 1)
(?, 1)
(?, 1)
(?, 1)
(?, 1)
(?, 1)
(?, 1)
next shape (?, 25, 1, 3)
<class 'tensorflow.python.framework.ops.Tensor'>
(?, 3, 25, 30)
(?, 1)
(?, 1)
(?, 1)
(?, 1)
(?, 1)
(?, 1)
(?, 1)
(?, 1)
(?, 1)
(?, 1)
(?, 1)
(?, 1)
(?, 1)
(?, 1)
(?, 1)
(?, 1)
(?, 1)
(?, 1)
(?, 1)
(?, 1)
next shape (?, 20, 1, 3)
<class 'tensorflow.python.framework.ops.Tensor'>
(?, 3, 20, 30)
(?, 1)
(?, 1)
(?, 1)
(?, 1)
(?, 1)
(?, 1)
(?, 1)
(?, 1)
(?, 1)
(?, 1)
(?, 1)
(?, 1)
(?, 1)
(?, 1)
(?, 1)
(?, 1)
(?, 1)
(?, 1)
(?, 1)
(?, 1)
next shape (?, 20, 1, 3)
epoch 0:
the times of training is 0, and the loss is 0.69213676
the times of training is 100, and the loss is 0.1608222
the times of training is 200, and the loss is 0.1527671
the times of training is 300, and the loss is 0.15371826
epoch 1:
the times of training is 0, and the 